# Prototypes and Criticisms on Fashion MNIST

Note that this cannot run on colab free tier due to memory constraints.

Based on the MMD-Critic algorithm by Kim, Been, Rajiv Khanna, and Oluwasanmi O. Koyejo. “Examples are not enough, learn to criticize! Criticism for interpretability.”

Code Reference: rajivak@utexas.edu


# Load Packages

In [1]:
import numpy as np
from torch.nn.functional import interpolate
import torch
import torch.nn.functional as F
import torchvision
import torchvision.transforms as T
from torchvision import models

from PIL import Image
from IPython.display import Image
import matplotlib.pyplot as plt
from matplotlib import gridspec

import sys
import math
import os
import json
import argparse
from pylab import *

from tensorflow import keras
from tensorflow.keras.datasets import fashion_mnist

from sklearn.metrics.pairwise import rbf_kernel

import warnings
warnings.filterwarnings('ignore')

# Load Fashion MNIST and transform dataset

In [2]:
num_classes = 10
input_shape = (28, 28, 1)

(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

X_train = X_train.astype("float32") / 255
X_test = X_test.astype("float32") / 255

X_tr = torch.from_numpy(X_train)
X_tr = interpolate(X_tr,scale_factor=0.572).permute(0, 2, 1)
X_tr = interpolate(X_tr,scale_factor=0.572).permute(0, 2, 1)
X_train = X_tr

X_tr = torch.from_numpy(X_test)
X_tr = interpolate(X_tr,scale_factor=0.572).permute(0, 2, 1)
X_tr = interpolate(X_tr,scale_factor=0.572).permute(0, 2, 1)
X_test = X_tr

print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)


X_train shape: torch.Size([60000, 16, 16])
X_test shape: torch.Size([10000, 16, 16])


# Data Class Definition

In [3]:
class Data:
    X = None # n * d
    y = None # n
    gamma = None
    kernel  = None # n* n

    def __init__(self):
        pass

    def splittraintest(self, testpercent):
        ntest = int(np.shape(self.X)[0] * testpercent/100.0)
        testindices = np.random.choice(np.shape(self.X)[0], ntest, replace=False)
        self.testX = self.X[testindices, :]
        self.testy = self.y[testindices]
        trainindices = np.setdiff1d(np.arange(np.shape(self.X)[0]), testindices)
        self.X = self.X[trainindices,:]
        self.y = self.y[trainindices]


    def subset(self, i):
        return np.where(y==i)[0]

    def load_data(self, X, y, testset, testy, gamma=None, docalkernel=False, savefile=None, dobin=False):
      self.X = X
      if dobin:
          bins = [-1.0, -0.67, -0.33, 0, 0.33, 0.67, 1.0]
          # bins = [-1.0, 0, 1.0]
          binned  = np.digitize(self.X, bins )
          self.X=np.array([bins[binned[i, j] - 1] for i in range(np.shape(self.X)[0]) for j in range(np.shape(self.X)[1])]).reshape(np.shape(self.X))

      self.y = y
      if testset is not None:
          self.testX = testset
          if dobin:
              bins = [-1.0, -0.67, -0.33, 0, 0.33, 0.67, 1.0]
              binned = np.digitize(self.testX, bins)
              self.testX = np.array([bins[binned[i, j] - 1] for i in range(np.shape(self.testX)[0]) for j in range(np.shape(self.testX)[1])]).reshape(np.shape(self.testX))

          self.testy = testy
 
      self.gamma = gamma
      self.kernel = rbf_kernel(self.X, gamma=gamma)

    def load_var(self, trainset, trainy, testset, testy, gamma=None, docalkernel=False, savefile=None, dobin=False):
         self.load_data(trainset, trainy, testset, testy, gamma, docalkernel, savefile, dobin)

    def calculate_kernel(self, g=None):
        if g is None:
            if self.gamma is None:
                print("gamma not provided!")
                exit(1)
            else:
                self.kernel = rbf_kernel(self.X, gamma=self.gamma)
        else:
            self.kernel = rbf_kernel(self.X, gamma=g)

    def calculate_kernel_individual(self, g=None):
        touseg = g
        if touseg is None:
            touseg = self.gamma
        if touseg is None:
            print("gamma not provided!")
            exit(1)
        self.kernel = np.zeros((np.shape(self.X)[0], np.shape(self.X)[0]) )
        sortind = np.argsort(self.y)
        self.X = self.X[sortind, :]
        self.y = self.y[sortind]

        for i in np.arange(10):
            j = i
            ind = np.where(self.y == i)[0]
            startind = np.min(ind)
            endind = np.max(ind)+1
            self.kernel[startind:endind, startind:endind ] = rbf_kernel(self.X[startind:endind, :], gamma=self.gamma)

    def loadstate(self,filename):
        temp = np.load(filename)
        self.X = temp['X']
        self.y = temp['y']
        self.gamma = temp['gamma']
        self.kernel = temp['kernel']

    def setgamma(self, newgamma):
        if self.kernel is not None:
            temp = np.log(self.kernel)
            temp = temp * newgamma/self.gamma
            self.kernel = np.exp(temp)
        self.gamma = newgamma
        if self.kernel is None:
            self.calculate_kernel()

    def savestate(self, outpfile):
        np.savez(file=outpfile, X=self.X, y=self.y, gamma=self.gamma, kernel=self.kernel)

    def rbf(v1,v2):
        dd = v1 - v2
        res = - self.gamma * np.dot(dd,dd)
        return math.exp(res)

    def getsim(self, i, j):
        if kernel is not None:
            return self.kernel[i,j]
        else:
            return self.rbf(X[i,:], X[j,:])

# MMD-Critic

In [4]:
def select_criticism_regularized(K, selectedprotos, m, reg='logdet', is_K_sparse=True):

    n = np.shape(K)[0]
    if reg in ['None','logdet','iterative']:
        pass
    else:
        print("wrong regularizer :" + reg)
        exit(1)
    options = dict()

    selected = np.array([], dtype=int)
    candidates2 = np.setdiff1d(range(n), selectedprotos)
    inverse_of_prev_selected = None  # should be a matrix

    if is_K_sparse:
        colsum = np.array(K.sum(0)).ravel()/n
    else:
        colsum = np.sum(K, axis=0)/n

    for i in range(m):
        maxx = -sys.float_info.max
        argmax = -1
        candidates = np.setdiff1d(candidates2, selected)

        s1array = colsum[candidates]

        temp = K[selectedprotos, :][:, candidates]
        if is_K_sparse:
            s2array = temp.sum(0)
        else:
            s2array = np.sum(temp, axis=0)

        s2array = s2array / (len(selectedprotos))

        s1array = np.abs(s1array - s2array)
        if reg == 'logdet':
            if inverse_of_prev_selected is not None: # first call has been made already
                temp = K[selected, :][:, candidates]
                if is_K_sparse:
                    temp2 = temp.transpose().dot(inverse_of_prev_selected)
                    regularizer = temp.transpose().multiply(temp2)
                    regcolsum = regularizer.sum(1).ravel()# np.sum(regularizer, axis=0)
                    regularizer = np.abs(K.diagonal()[candidates] - regcolsum)

                else:
                    # hadamard product
                    temp2 = np.array(np.dot(inverse_of_prev_selected, temp))
                    regularizer = temp2 * temp
                    regcolsum = np.sum(regularizer, axis=0)
                    regularizer = np.log(np.abs(np.diagonal(K)[candidates] - regcolsum))
                s1array = s1array + regularizer
            else:
                if is_K_sparse:
                    s1array = s1array - np.log(np.abs(K.diagonal()[candidates]))
                else:
                    s1array = s1array - np.log(np.abs(np.diagonal(K)[candidates]))
        argmax = candidates[np.argmax(s1array)]
        maxx = np.max(s1array)

        selected = np.append(selected, argmax)
        if reg == 'logdet':
            KK = K[selected,:][:,selected]
            if is_K_sparse:
                KK = KK.todense()

            inverse_of_prev_selected = np.linalg.inv(KK) # shortcut
        if reg == 'iterative':
            selectedprotos = np.append(selectedprotos, argmax)

    return selected


def greedy_select_protos(K, candidate_indices, m, is_K_sparse=False):

    if len(candidate_indices) != np.shape(K)[0]:
        K = K[:,candidate_indices][candidate_indices,:]

    n = len(candidate_indices)

    # colsum = np.array(K.sum(0)).ravel() # same as rowsum
    if is_K_sparse:
        colsum = 2*np.array(K.sum(0)).ravel() / n
    else:
        colsum = 2*np.sum(K, axis=0) / n

    selected = np.array([], dtype=int)
    value = np.array([])
    for i in range(m):
        maxx = -sys.float_info.max
        argmax = -1
        candidates = np.setdiff1d(range(n), selected)

        s1array = colsum[candidates]
        if len(selected) > 0:
            temp = K[selected, :][:, candidates]
            if is_K_sparse:
                # s2array = temp.sum(0) *2
                s2array = temp.sum(0) * 2 + K.diagonal()[candidates]

            else:
                s2array = np.sum(temp, axis=0) *2 + np.diagonal(K)[candidates]

            s2array = s2array/(len(selected) + 1)

            s1array = s1array - s2array

        else:
            if is_K_sparse:
                s1array = s1array - (np.abs(K.diagonal()[candidates]))
            else:
                s1array = s1array - (np.abs(np.diagonal(K)[candidates]))

        argmax = candidates[np.argmax(s1array)]

        selected = np.append(selected, argmax)
        KK = K[selected, :][:, selected]
        if is_K_sparse:
            KK = KK.todense()

        inverse_of_prev_selected = np.linalg.inv(KK)  # shortcut

    return candidate_indices[selected]


def format_numsel(numsel):
    ss = ''
    for i,j in enumerate(numsel):
        ss = ss + " %d:%d " %(i,j)
    return ss

def get_train_testindices(n, ntest, seed):
    np.random.seed(seed)
    testindices = np.random.choice(n,ntest,replace=False)
    trainindices = np.setdiff1d( range(n), testindices)
    return trainindices, testindices

def exit(str):
    print(str)
    exit(1)


def dir_exists(filename):
    """Creates the directory of a file if the directory does not exist.
    
    Raises:
      IOError: If the directory could not be created (and the directory does not
          exist). This may be due to for instance permissions issues or a race
          condition in which the directory is created right before makdirs runs.
    """
    dir = os.path.dirname(filename)
    if not os.path.exists(dir):
        os.makedirs(dir)
        
def plotfigs2(xx, selectedy, fileprefix=None, printselectionnumbers = False):
    num_selected = np.array([0] * 10)
    for ii in range(10):
        num_selected[ii] = len(np.where(selectedy == (ii + 1))[0])
        print(ii, num_selected[ii])

    totm = np.shape(xx)[0]
    print("number of images being printed %d" %totm)
    perpic_m = 40
    begin_at = 0
    counter = 0
    perrow = 8

    while counter < int(totm/perpic_m) + 1:

        counter += 1
        offset = 0
        if begin_at == 0:
            offset = 5 # for text about number of protos/crits of each type
        if not printselectionnumbers:
            offset = 0

        # m=m+offset  # for num_selected
        gs = gridspec.GridSpec(int(perpic_m/perrow+offset/perrow), perrow, wspace=0.0, hspace=0.0)
        fig = plt.figure()

        if begin_at == 0 and printselectionnumbers:
            ax=fig.add_subplot(gs[0,:])
            ax.text(0.1,0.5,format_numsel(num_selected))
            ax.axis('off')

        endd = begin_at + offset+ perpic_m
        if endd-offset > totm:
            endd = totm +offset
        for i in np.array(range(begin_at + offset, endd)):
            ax = fig.add_subplot(gs[i - begin_at])
            ax.imshow(xx[i - offset, :].reshape((16, 16)))
            ax.axis('off')

        file = fileprefix+str(counter) + '.png'
        if file is not None:
            plt.savefig(file , dpi=2000)

        begin_at += perpic_m


def run(trainset, trainy, testset, testy, gamma, m, k, ktype, outfig, critoutfig):

    digitsdat = Data()
    digitsdat.load_var(trainset.reshape(60000,1,-1).squeeze(),
                       trainy=trainy,
                       testset=testset.reshape(10000,1,-1).squeeze(),
                       testy=testy,
                       gamma=gamma, 
                       docalkernel=False, 
                       savefile=None, 
                       dobin=False)

    if ktype == 0:
        digitsdat.calculate_kernel()
        print("Running Kernel type : global ")
    else:
        digitsdat.calculate_kernel_individual()
        print("Running Kernel type : local ")



    # selected = greedy_parallel(digitsdat.kernel, m)
    # print(np.sort(selected))
    selected = greedy_select_protos(digitsdat.kernel, np.array(range(np.shape(digitsdat.kernel)[0])), m)
    # print(np.sort(selected))
        # critselected = select_criticism(digitsdat.kernel, selected, k)
    selectedy = digitsdat.y[selected]
    sortedindx = np.argsort(selectedy)
    critselected= None

    if outfig is not None:
        plotfigs2(digitsdat.X[selected[sortedindx], :], selectedy[sortedindx], outfig)


    if k > 0:
        critselected = select_criticism_regularized(digitsdat.kernel, selected, k, is_K_sparse=False, reg='logdet')
        reg = 'logdet'
        critselectedy = digitsdat.y[critselected]
        critsortedindx = np.argsort(critselectedy)

        if critoutfig is not None:
            plotfigs2(digitsdat.X[critselected[critsortedindx], :], critselectedy[critsortedindx], critoutfig+reg)

    print("prototypes = ",selected)
    print()
    print("criticisms = ",critselected)
    return selected, critselected, digitsdat

In [5]:
data_prefix = '.' 
output_prefix = '.' 
gamma = 0.026 # k

m = 400  # total number of prototypes to select

do_output_pics = 1
kernel_type = 1 # 1 for local, 0 for global

ioff()

k = 400 # number of criticisms

outfig = os.path.join(output_prefix, 'images/%d/protos' % m)
critoutfig = os.path.join(output_prefix, 'images/%d/crit' % m)

dir_exists(outfig)

selected, critselected, digitsdat = run(X_train,
                                        y_train,
                                        X_test,
                                        y_test,
                                        gamma,
                                        m,
                                        k,
                                        kernel_type,
                                        outfig,
                                        critoutfig)

Running Kernel type : local 
0 40
1 39
2 40
3 41
4 39
5 40
6 41
7 40
8 40
9 0
number of images being printed 400
0 18
1 55
2 26
3 33
4 58
5 55
6 13
7 65
8 22
9 0
number of images being printed 400
prototypes =  [47473  9056 23634  4556 34300 25135 59002 13119 36546 53465 21054  9601
 56544 47894 24287 17776 30465  3855 37519 52633 22999  9517 25326 16431
 56503 42317  4060 50036 30377 36308 19899 13094 28063  3928  9047 57775
 39696 43066 32760 49915 22956 17758 56154 28409  6308  1324 40265 44374
 53360 34568 22138 13485  4973 36748 55566  7255 24256 42659 51911 33700
 22296 39030  1137 43056  6146 15213 56706 49653 28893 33209 37267 19470
 57130 42352  4158 10179 24116 15639 51414 30301 19663 37553  5465 56749
 42257  6234 50289 28718 13657 33207 40672  9602 23044  3342 55414 46348
 52631 28206 30237 12251 37163 19968 26097  6911 43762 12341 57976  3929
 50202 30347 36905 29515 23446 11850 12964 44418 35804  2682 56902 49431
 37071 25788  8192 23970  5341 13559 42474 53784 35699 5887

In [6]:
display(Image(filename='images/%d/protos1.png' % m))
display(Image(filename='images/%d/critlogdet1.png' % m))